In [1]:
from dotenv import load_dotenv
import dspy
import mlflow

from agentic_system.agents import (
    CFEfficacyAgent,
    ToxicityScreeningAgent,
    CompoundPrioritizationAgent,
)

In [4]:
# NOTE: Start MLflow server with:
# mlflow server --backend-store-uri sqlite:///mydb.sqlite
# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Create a unique name for your experiment.
mlflow.set_experiment("Optimize Efficacy Tools")
mlflow.autolog()
mlflow.tracing.disable_notebook_display()

load_dotenv("../.env")
dspy.configure(
    lm=dspy.LM("gemini/gemini-2.5-pro", temperature=0.9, cache=True, rollout_id=1)
)

COMPOUND = "Luminespib"

2025/09/17 15:51:33 INFO mlflow.tracking.fluent: Autologging successfully enabled for dspy.
2025/09/17 15:51:33 INFO mlflow.tracking.fluent: Autologging successfully enabled for litellm.
2025/09/17 15:51:33 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.


In [5]:
efficacy_agent = CFEfficacyAgent(max_iters=3)
# efficacy_agent.agent.react.signature = efficacy_agent.agent.react.signature.insert(
#     0,
#     "tool_expected_info",
#     dspy.OutputField(
#         desc="TRUE or FALSE: <more info>. If FALSE, describe what information (ex databases, literature, etc) was desired from tool. Do not say FALSE if tool did not return any information (ex due to no information available), only say FALSE if tool returned information that was not useful or other information was desired from this tool call.\n\n"
#     ),
#     type_=str,
# )
efficacy_results = efficacy_agent(compound_name=COMPOUND)
print(efficacy_results)

Prediction(
    trajectory={'thought_0': 'To estimate the efficacy of Luminespib for reversing the failing cardiac fibroblast phenotype, I first need to understand its biological function. My initial step is to identify the compound in a chemical database to retrieve its mechanism of action and other relevant data. I will start by searching for its ChEMBL ID.', 'tool_name_0': 'CHEMBL__search_chembl_id', 'tool_args_0': {'goal': 'Find the ChEMBL ID for Luminespib to investigate its mechanism of action and bioactivities.', 'query': 'Luminespib'}, 'observation_0': 'Luminespib has two ChEMBL IDs: CHEMBL3137334 (LUMINESPIB MESYLATE) and CHEMBL252164 (LUMINESPIB).', 'thought_1': 'I have successfully identified two ChEMBL IDs for Luminespib. I will proceed with `CHEMBL252164`, which represents the parent compound, to investigate its mechanism of action. Understanding how Luminespib works at a molecular level is the next critical step in forming a hypothesis about its potential to reverse the f

In [9]:
dspy.configure(
    lm=dspy.LM("gemini/gemini-2.5-pro", temperature=0.9, cache=False, rollout_id=1)
)

efficacy_agent = CFEfficacyAgent(max_iters=3)
# efficacy_agent.agent.react.signature = efficacy_agent.agent.react.signature.insert(
#     0,
#     "tool_expected_info",
#     dspy.OutputField(
#         desc="TRUE or FALSE: <more info>. If FALSE, describe what information (ex databases, literature, etc) was desired from tool. Do not say FALSE if tool did not return any information (ex due to no information available), only say FALSE if tool returned information that was not useful or other information was desired from this tool call.\n\n"
#     ),
#     type_=str,
# )
efficacy_results = efficacy_agent(compound_name=COMPOUND)
print(efficacy_results)

Prediction(
    trajectory={'thought_0': 'I need to estimate the efficacy of Luminespib for reversing the failing cardiac fibroblast phenotype. My first step is to identify the compound and its primary mechanism of action. I will start by searching for Luminespib in the ChEMBL database to obtain its ChEMBL ID, which will allow me to retrieve more detailed information about its bioactivities and mechanism of action.', 'tool_name_0': 'CHEMBL__search_chembl_id', 'tool_args_0': {'goal': 'Find the ChEMBL ID for the compound Luminespib to initiate research into its mechanism of action and potential effects on cardiac fibroblasts.', 'query': 'Luminespib'}, 'observation_0': 'The ChEMBL IDs for Luminespib are CHEMBL3137334 (LUMINESPIB MESYLATE) and CHEMBL252164 (LUMINESPIB).', 'thought_1': 'I have successfully retrieved the ChEMBL IDs for Luminespib. I will use "CHEMBL252164" as it appears to be the parent compound. My next step is to investigate the mechanism of action of Luminespib. Understan

In [6]:
# tox_agent = ToxicityScreeningAgent(max_iters=12)
# tox_results = tox_agent(compound_name=COMPOUND)
# print(tox_results)

In [7]:
# prioritization_agent = CompoundPrioritizationAgent()
# prioritization_results = prioritization_agent(compound_name=COMPOUND)
# print(prioritization_results)